# Dollar bars vs Volume bars Sunny
Purpose: To determine whether we should use dollar bars or volume bars for the rest of this project. First, we will find metrics that are important, then find out how to implement them, then come to a conclusion on which bars perform the best. 

Helpful resources:
- https://towardsdatascience.com/advanced-candlesticks-for-machine-learning-ii-volume-and-dollar-bars-6cda27e3201d
- Advances in Financial Machine Learning from 2.3.1 to 2.3.2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
db = pd.read_csv("../data/BTCUSDT-dollar-bars.csv")
vb = pd.read_csv("../data/BTCUSDT-volume-bars.csv")
tb = pd.read_csv("../data/discarded/BTCUSDT-minute-target-ui.csv")